In [1]:
import pandas as pd
from urllib.parse import urlencode
import requests
from requests import Response
import plotly.graph_objects as go
import sqlalchemy
from datetime import datetime, date, time
import os, sys
import datetime as dt
from dateutil.relativedelta import relativedelta
from math import ceil
import numpy as np

In [2]:
sys.path.append(os.path.abspath('.'))

In [3]:
import foward_test_valuation_strategy as common

In [4]:
keys = ['total equity', 'interest expenses', 'fixed assets', 'net accounting profit/(loss) before tax']

list_stock_link = 'https://trading.vcsc.com.vn/restapi/api/v1/market/stock/listed?marketType=HOSE&securitiesType=ALL&fetchCount=1000'
hose_list = pd.DataFrame(common.get_rest(url=list_stock_link, body={}))
hose_list['len'] = hose_list['code'].apply(lambda x: len(x))
hose_list = hose_list[hose_list['len'] == 3].code.tolist()


In [ ]:
# magic1_universe = pd.read_csv('magic1-universe.csv')['code'].to_list()
lst_type = common.get_group_market_cap(date='2021-02-17', lst_code=hose_list)

# If you don't have finincial_data.csv, run this code.

In [ ]:
# access_tok = common.login()

In [ ]:
# data = pd.DataFrame()
# for key in keys:
#     for year in range(2014, 2021):
#         for quarter in range(1, 5):
#             df = common.query_financial_data(year=year, quarter=quarter, list_stock=hose_list, type=key, access_token=access_tok)
#             if len(df) > 0:
#                 df = pd.melt(df, id_vars=['code', 'quarter', 'year'], value_vars=key)
#             data = pd.concat([data, df])
#             print('KEY: {} ====== YEAR: {} ===== QUAR: {}'.format(key, year, quarter))
#     print('DONEEEEEEE')
# data

In [ ]:
# data.to_csv('financial_data.csv', index=None)

# START HEREEEE

In [ ]:
data = pd.read_csv('financial_data.csv')

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
equity = data[data['variable']=='total equity']
interest = data[data['variable']=='interest expenses']
asset = data[data['variable']=='fixed assets']
ebt = data[data['variable']=='net accounting profit/(loss) before tax']

In [ ]:
equity = equity.drop(columns=['variable']).rename(columns={'value':'total equity'})
interest = interest.drop(columns=['variable']).rename(columns={'value':'interest expenses'})
asset = asset.drop(columns=['variable']).rename(columns={'value':'fixed assets'})
ebt = ebt.drop(columns=['variable']).rename(columns={'value':'net accounting profit/(loss) before tax'})

In [ ]:
data = pd.merge(equity, interest, on=['code', 'quarter', 'year'], how='outer')
data = pd.merge(data, asset, on=['code', 'quarter', 'year'], how='outer')
data = pd.merge(data, ebt, on=['code', 'quarter', 'year'], how='outer')

In [ ]:
data = data.sort_values(by=['code', 'year', 'quarter']).reset_index(drop=True)

In [ ]:
data = data.rename(columns={'net accounting profit/(loss) before tax': 'ebt'})

In [ ]:
data['ebit'] = data['ebt'] + data['interest expenses']
data.loc[data['ebit'].isna(), 'ebit'] = data['ebt']

In [ ]:
data

In [ ]:
price = common.get_stock_price(stock_arr=hose_list, 
                               to_date='2021-02-18', 
                               from_date='2015-01-01', pivot_type=False, price=True)

In [ ]:
oi = common.get_stock_oi(stock_arr=hose_list, to_date='2021-02-18', from_date='2015-01-01', pivot_type=False)

In [ ]:
price_oi = pd.merge(price, oi, on=['date', 'code'], how='left')

In [ ]:
price_oi['marcap'] = price_oi['close'] * 1000 * price_oi['oi']

In [ ]:
price_oi['date'] = price_oi['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
price_oi['month'] = price_oi['date'].dt.month

In [ ]:
price_oi.loc[(price_oi['month']!=price_oi['month'].shift(-1))|(
            price_oi['code']!=price_oi['code'].shift(-1)), 'choose'] = 1
price_oi_monthly = price_oi[price_oi['choose']==1].reset_index(drop=True).drop(columns=['choose', 'month'])

In [ ]:
price_oi_monthly = pd.merge(price_oi_monthly, lst_type, on=['code'], how='left')

In [ ]:
price_oi_monthly['date_lag_x'] = price_oi_monthly['date'].apply(lambda x: x-relativedelta(months=5))
price_oi_monthly['year'] = price_oi_monthly['date_lag_x'].dt.year
price_oi_monthly['quarter'] = price_oi_monthly['date_lag_x'].dt.month.apply(lambda x: ceil(x/3))


In [ ]:
price_oi_fin_monthly = pd.merge(price_oi_monthly, data, on=['code', 'year', 'quarter'])

In [ ]:
price_oi_fin_monthly['roic'] = price_oi_fin_monthly['ebit'] / (price_oi_fin_monthly['total equity']
                                                               +price_oi_fin_monthly['fixed assets'])

price_oi_fin_monthly['earnings yield'] = price_oi_fin_monthly['ebit'] / price_oi_fin_monthly['marcap']

In [ ]:
price_oi_fin_monthly['year-quarter'] = price_oi_fin_monthly['year'].apply(lambda x: str(x)) + (
                                                                    price_oi_fin_monthly['quarter'].apply(lambda x: str(x)))

In [ ]:
price_oi_fin_monthly[price_oi_fin_monthly['code']=='AAA']

# Filter by Group

In [ ]:
price_oi_fin_monthly_filtered = price_oi_fin_monthly[(price_oi_fin_monthly['type']=='midCap')]

# price_oi_fin_monthly_filtered = price_oi_fin_monthly.copy(deep=True)

# Continue

In [ ]:
price_oi_fin_monthly_filtered

In [ ]:
roic = pd.pivot(price_oi_fin_monthly_filtered, 
                index='date', columns='code', values='roic').dropna(how='all', axis=1)

In [ ]:
earnings = pd.pivot(price_oi_fin_monthly_filtered, 
                    index='date', columns='code', values='earnings yield').dropna(how='all', axis=1)

In [ ]:
roic_ranking = roic.copy(deep=True)
earnings_ranking = earnings.copy(deep=True)
for col in roic_ranking.columns:
    roic_ranking[col] = np.nan
    earnings_ranking[col] = np.nan

In [ ]:
roic_ranking = roic.rank(axis=1, ascending=False)
earnings_ranking = earnings.rank(axis=1, ascending = False)

In [ ]:
combined_ranks = (roic_ranking + earnings_ranking)/2

combined_ranks = combined_ranks.rank(axis=1, ascending=True)

In [ ]:
combined_ranks

In [ ]:
price_monthly_pivot = price_oi_fin_monthly_filtered.pivot(index='date', columns='code', values='close')
monthly_return = price_monthly_pivot/price_monthly_pivot.shift(1)-1

In [ ]:
### CHECK PERFORMANCE OF THE TOP OF SPECIFIC STOCKS (TOP 15 STOCKS, 10 STOCKS, ...)

positions = monthly_return.copy(deep=True)
for col in positions.columns:
    positions[col] = np.nan
    
for col in positions.columns:
    if col in combined_ranks.columns:
        positions[col] = combined_ranks[col].apply(lambda x: 1 if x<=15 else np.nan)
    else:
        positions = positions.drop(columns=col)
        
profit = monthly_return * positions.shift(1)

profit['avgProfit'] = profit.mean(axis=1)

profit['cumulative'] = (profit['avgProfit'] + 1).cumprod()

profit['numHolding'] = profit.count(axis=1)

import plotly.graph_objects as go

benchmark = (monthly_return.mean(axis=1)+ 1).cumprod().tolist()
fig = go.Figure()

import plotly.graph_objects as go
cumulative = profit['cumulative'].to_list()
benchmark = (monthly_return.mean(axis=1)+ 1).cumprod().tolist()
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=profit.index.tolist(), y=cumulative,
               mode='lines',
               name='cumulative'))
fig.add_trace(
    go.Scatter(x=profit.index.tolist(), y=benchmark,
               mode='lines',
               name='benchmark'))

fig.show()


# CHECK PERFORMANCE OF THE TOP BY % (TOP 10%, 20%,...)

In [ ]:
combined_ranks['count'] = combined_ranks.count(axis='columns')

for col in combined_ranks.columns[:-1]:
    combined_ranks[col] = combined_ranks[col]/combined_ranks['count']
    

In [ ]:


lst_rank = [10,20,30,40,50,60,70,80,90,100]

result = positions.reset_index()[['date']].set_index('date')

for r in lst_rank:
    positions = monthly_return.copy(deep=True)
    for col in positions.columns:
        positions[col] = np.nan
    profit = positions.copy(deep=True)
    for col in positions.columns:
        if col in combined_ranks.columns:
            positions[col] = combined_ranks[col].apply(lambda x: 1 if x<=r*0.01 else np.nan)
        else:
            positions = positions.drop(columns=col)
    profit = monthly_return * positions.shift(1)
    profit['avgProfit'] = profit.mean(axis=1)
    profit['numHolding'] = profit.count(axis=1)
    
    result[str(r)] = profit['avgProfit']
    
    
    print('Top {}'.format(str(r)))
    print(profit['numHolding'].max())
    
for r in lst_rank:
    if r > 10:
        result['avgProfit'+ str(r)] = result[str(r)]*r/10 - result[str(r-10)]*(r-10)/10
    else:
        result['avgProfit'+ str(r)] = result[str(r)]
    
    result['cumulative' + str(r)] = (result['avgProfit'+ str(r)]+1).cumprod()
        
#     print('Top {} === Profit {} === Max Holding {}'.format(str(r*10), ))

# profit